In [106]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

Read csv

In [107]:
SUMMER = pd.read_csv('summer.csv')

In [108]:
SUMMER.head(10)

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver
5,1896,Athens,Aquatics,Swimming,"CHOROPHAS, Efstathios",GRE,Men,1200M Freestyle,Bronze
6,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,1200M Freestyle,Gold
7,1896,Athens,Aquatics,Swimming,"ANDREOU, Joannis",GRE,Men,1200M Freestyle,Silver
8,1896,Athens,Aquatics,Swimming,"CHOROPHAS, Efstathios",GRE,Men,400M Freestyle,Bronze
9,1896,Athens,Aquatics,Swimming,"NEUMANN, Paul",AUT,Men,400M Freestyle,Gold


Get sorted cycling medal count by country.

In [109]:
cycling = SUMMER.query('Sport == "Cycling"').groupby(['Country'], as_index=False)['Medal'].count().sort_values(by='Medal', ascending=False)

In [110]:
cycling.head(10)

,Country,Medal
16,GBR,145
14,FRA,136
21,ITA,115
1,AUS,76
41,USA,66
18,GER,65
28,NED,61
39,URS,51
10,DEN,43
3,BEL,40


Let's put this data on a map!

In [131]:
layout = dict(title=dict(text=('Olympic Cycling Medals Awarded by Country '
                             '1896-2012'),
                             x=0.5,
                             ),
              geo=dict(countrycolor='#444444',
                       showcountries=True,
                       oceancolor='deepskyblue',
                       showocean=True,
                       lakecolor='deepskyblue',
                       showlakes=True,
                       projection={'type': 'natural earth'},
                       )
              )

In [112]:
data = go.Choropleth(locations=cycling['Country'],
                     locationmode='ISO-3',
                     z=cycling['Medal'],
                     colorscale=px.colors.sequential.Oryel)

In [132]:
fig = go.Figure(data=data, layout=layout)
fig.show()

We can see that this doesn't look quite right. In Europe, Germany and the Netherlands, which are in the Top-10 from our cycling.head(10) selection above, don't reflect any medals on our map. There is an issue with the country codes! Using ISO-3, we need to replace some country codes in order to have them display correctly on our map.

In [114]:
cycling['Country'] = cycling['Country'].replace(
        ['GER', 'NED', 'DEN', 'SUI', 'RSA', 'GRE', 'LAT', 'POR', 'URU'],
        ['DEU', 'NLD', 'DNK', 'CHE', 'ZAF', 'GRC', 'LVA', 'PRT', 'URY']
        )

In [115]:
data = go.Choropleth(locations=cycling['Country'],
                     locationmode='ISO-3',
                     z=cycling['Medal'],
                     colorscale=px.colors.sequential.Oryel)

In [116]:
fig = go.Figure(data=data, layout=layout)
fig.show()

Now, that looks better!

In reading the plotly documentation, I discovered the scope arg in layout.geo. This gave me the idea to create a region selector for the final figure, to allow for an easy zoom-in to any continent.

In [117]:
def region_scopes(region: str):
    # Pass to geo for region selection on map.
    return dict(countrycolor='#444444',
                showcountries=True,
                oceancolor='deepskyblue',
                showocean=True,
                lakecolor='deepskyblue',
                showlakes=True,
                projection={'type': 'natural earth'},
                scope=region,
                )

Add region selector and display figure.

In [128]:
fig.update_layout(
        updatemenus=[
            dict(buttons=list([
                dict(label='World',
                     method='relayout',
                     args=['geo', region_scopes('world')]),
                dict(label='Africa',
                     method='relayout',
                     args=['geo', region_scopes('africa')]),
                dict(label='Asia',
                     method='relayout',
                     args=['geo', region_scopes('asia')]),
                dict(label='Europe',
                     method='relayout',
                     args=['geo', region_scopes('europe')]),
                dict(label='North America',
                     method='relayout',
                     args=['geo', region_scopes('north america')]),
                dict(label='South America',
                     method='relayout',
                     args=['geo', region_scopes('south america')])
            ]),
             x=0.2,
             y=1.15,
             direction='right'
            )
        ],
    )